In [1]:
import torch
import tensorflow
import gym
import keras
import random

import numpy as np
from gym import Env

from gym.spaces import Discrete, Box

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# get training envs
import pandas as pd
all_puzzles = pd.read_csv("sudoku.csv")
all_puzzles_df = pd.DataFrame(all_puzzles)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

import time

cuda


In [2]:
class board_player_env(Env):
    def __init__(self, game_no = np.random.randint(100000)):
        super(board_player_env, self).__init__()
        # creating the action space
        # self.game_no = game_no
        random.seed(time.perf_counter())
        game_no = np.random.randint(100000)
        
        print("game being played now:", game_no)
        self.action_space_list = ['up', 'down', 'left', 'right', 1, 2, 3, 4, 5, 6, 7, 8, 9]
        self.action_space = Discrete(len(self.action_space_list))
        # number of valid actions
        self.n_actions = len(self.action_space_list)
        self.n_features = 2

        # observation space
        self.observation_space = Box(low=0, high=9, shape=(9, 9), dtype=int)

        # initializing environment
        self.puzzle = all_puzzles_df.iloc[game_no]['quizzes']
        self.puzzle_arr = np.array(list(self.puzzle), dtype=int).reshape((9, 9))

        # initializing perfect solution
        self.solution = all_puzzles_df.iloc[game_no]['solutions']
        self.solution_arr = np.array(list(self.solution), dtype=int).reshape((9, 9))

        # initializing the agent at the first index
        self.agent = [0, 0]

        # initializing rewards
        self.rewards = 0
        self.done = False

    def render(self):
        print("\nsolved now: \n")
        print(self.puzzle_arr)

    def reset(self):
        # set the agent at first location
        # self.state = [0, 0]
        
        # maybe it shoulf be there/maybe not
        # self.rewards = 0
        # self.done = False
        return [0, 0]

    def step(self, action):

        # current agent location
        state = self.agent
        # print("self.agent: ", self.agent)
        # print("state: ", state[0])
        # print("action: ", action, type(action))

        # rewards at each step
        # reward += 1 for reaching empty
        # reward += 3 for filling one spot correctly
        # reward += 10 for jackpot of full puzzle
        # punishment += -2 for filling wrong

        # action 0, 1, 2, 3
        # imply only some filling in rules of the sudoku game, so that the board doesn't change
        if (action == 0):
            # print("\nup")
            if(state[0] > 0):
                self.agent = [state[0] - 1, state[1]]
                if(self.puzzle_arr[state[0] - 1, state[1]] == 0):
                    print("\nreward for up")
                    self.rewards += 1
                else:
                    print("\npunishment for up")
                    self.rewards -= 1

        elif (action == 1):
            # print("\ndown")
            if(state[0] < 8):
                self.agent = [state[0] + 1, state[1]]
                if(self.puzzle_arr[state[0] + 1, state[1]] == 0):
                    print("\nreward for down")
                    self.rewards += 1
                else:
                    print("\npunishment for down")
                    self.rewards -= 1

        elif (action == 2):
            # print("\nleft")
            if(state[1] > 0):
                self.agent = [state[0], state[1] - 1]
                if(self.puzzle_arr[state[0], state[1] - 1] == 0):
                    print("\nreward for left")
                    self.rewards += 1
                else:
                    print("\npunishment for left")
                    self.rewards -= 1

        elif (action == 3):
            # print("\nright")
            if(state[1] < 8):
                self.agent = [state[0], state[1] + 1]
                if(self.puzzle_arr[state[0], state[1] + 1] == 0):
                    print("\nreward for right")
                    self.rewards += 1
                else:
                    print("\npunishment for right")
                    self.rewards -= 1

        # action 5 means fill 2
        elif (action > 3):
            if(self.puzzle_arr[state[0], state[1]] == 0):
                if(self.action_space_list[action] == self.solution_arr[state[0], state[1]]):
                    self.puzzle_arr[state[0], state[1]] = self.action_space_list[action]
                    print("\nreward for action:", self.action_space_list[action])
                    self.rewards += 3
                else:
                    print("\npunishment for action:", self.action_space_list[action])
                    self.rewards -= 2

        if(np.array_equal(self.solution_arr, self.puzzle_arr)):
            self.rewards += 10
            self.done = True
            self.render()
            # maybe/maybe not
            self.__init__()
            

        return self.agent, self.rewards, bool(self.done), {}

In [3]:
def build_model(states, actions):
    model = Sequential()
    # model.add(Reshape(target_shape=(actions,), input_shape=states))
    model.add(Dense(24, activation='relu', input_shape=(1, 2, )))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions))
    model.add(Flatten())
    return model

In [4]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-3)
    return dqn

In [5]:
env = board_player_env()
states = env.observation_space.shape
actions = env.action_space.n

game being played now: 77410


del model

In [6]:
model = build_model(states, actions)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 24)             72        
_________________________________________________________________
dense_1 (Dense)              (None, 1, 24)             600       
_________________________________________________________________
dense_2 (Dense)              (None, 1, 13)             325       
_________________________________________________________________
flatten (Flatten)            (None, 13)                0         
Total params: 997
Trainable params: 997
Non-trainable params: 0
_________________________________________________________________


In [8]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(board_player_env(), nb_steps=50000, visualize=False, verbose=1)

game being played now: 75072
Training for 50000 steps ...
Interval 1 (0 steps performed)


/home/vanillaskies/anaconda3/envs/mne/lib/python3.9/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



punishment for action: 7
    1/10000 [..............................] - ETA: 1:07:34 - reward: -2.0000
punishment for action: 5

punishment for action: 2

reward for action: 3

punishment for down

punishment for up


/home/vanillaskies/anaconda3/envs/mne/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


   12/10000 [..............................] - ETA: 22:42 - reward: -4.4167  
punishment for down
   18/10000 [..............................] - ETA: 15:15 - reward: -4.6667
reward for down

punishment for action: 1

punishment for action: 4

reward for right

punishment for action: 9
   24/10000 [..............................] - ETA: 11:38 - reward: -5.3750
punishment for down

reward for down
   29/10000 [..............................] - ETA: 9:52 - reward: -6.3103 
punishment for action: 9

reward for action: 2

reward for down
   36/10000 [..............................] - ETA: 8:09 - reward: -6.8889
punishment for action: 2

punishment for action: 7

punishment for action: 7

punishment for action: 4

punishment for action: 9

punishment for action: 2
   42/10000 [..............................] - ETA: 7:10 - reward: -8.0476
punishment for action: 8

punishment for action: 2

punishment for action: 4

punishment for action: 5

punishment for action: 4
   47/10000 [..............

...

 8091/10000 [=======================>......] - ETA: 16s - reward: -1077.9204
reward for up

punishment for up

punishment for up
 8098/10000 [=======================>......] - ETA: 16s - reward: -1078.3857
punishment for down

punishment for right
 8105/10000 [=======================>......] - ETA: 16s - reward: -1078.8524
punishment for left

punishment for right
 8112/10000 [=======================>......] - ETA: 16s - reward: -1079.3199
punishment for down

punishment for up
 8118/10000 [=======================>......] - ETA: 15s - reward: -1079.7214
punishment for left

punishment for up

punishment for right
 8125/10000 [=======================>......] - ETA: 15s - reward: -1080.1910
punishment for up

punishment for right

punishment for left

punishment for down
 8131/10000 [=======================>......] - ETA: 15s - reward: -1080.5954
punishment for left
 8138/10000 [=======================>......] - ETA: 15s - reward: -1081.0681
punishment for down

reward for down

punishme

 8614/10000 [========================>.....] - ETA: 11s - reward: -1114.7075
punishment for right
 8621/10000 [========================>.....] - ETA: 11s - reward: -1115.2249
punishment for down
 8627/10000 [========================>.....] - ETA: 11s - reward: -1115.6679
punishment for right
 8633/10000 [========================>.....] - ETA: 11s - reward: -1116.1114
punishment for left

punishment for up
 8652/10000 [========================>.....] - ETA: 11s - reward: -1117.5158
punishment for left

punishment for up
 8658/10000 [========================>.....] - ETA: 11s - reward: -1117.9589
punishment for right

punishment for left
 8664/10000 [========================>.....] - ETA: 11s - reward: -1118.4027
punishment for left

punishment for down

punishment for up

punishment for up
 8670/10000 [=========================>....] - ETA: 11s - reward: -1118.8481
punishment for up

punishment for up
 8674/10000 [=========================>....] - ETA: 11s - reward: -1119.1462
punishmen

 9142/10000 [==========================>...] - ETA: 7s - reward: -1155.4562
punishment for left

punishment for down

punishment for down
 9149/10000 [==========================>...] - ETA: 7s - reward: -1156.0189
punishment for down
 9156/10000 [==========================>...] - ETA: 7s - reward: -1156.5817
punishment for left

punishment for up

punishment for left
 9163/10000 [==========================>...] - ETA: 7s - reward: -1157.1455
punishment for right

punishment for up

punishment for left
 9170/10000 [==========================>...] - ETA: 7s - reward: -1157.7109
punishment for down

punishment for right
 9177/10000 [==========================>...] - ETA: 7s - reward: -1158.2772
punishment for up

punishment for down
 9184/10000 [==========================>...] - ETA: 6s - reward: -1158.8440
punishment for up

punishment for down

punishment for down
 9191/10000 [==========================>...] - ETA: 6s - reward: -1159.4125
punishment for left
 9198/10000 [===============

 9664/10000 [===========================>..] - ETA: 2s - reward: -1199.0896
reward for down

punishment for action: 9

punishment for action: 3

punishment for action: 7

punishment for action: 8

punishment for action: 4

reward for action: 1

solved now: 

[[4 6 9 2 3 8 5 1 7]
 [7 3 2 1 9 5 6 8 4]
 [5 8 1 7 4 6 3 2 9]
 [6 2 4 9 5 3 1 7 8]
 [8 9 5 6 1 7 4 3 2]
 [1 7 3 4 8 2 9 5 6]
 [3 4 6 8 7 1 2 9 5]
 [2 1 7 5 6 9 8 4 3]
 [9 5 8 3 2 4 7 6 1]]
game being played now: 79078
 9671/10000 [============================>.] - ETA: 2s - reward: -1199.4824
punishment for down

reward for right

punishment for action: 9

reward for down

punishment for action: 5
 9678/10000 [============================>.] - ETA: 2s - reward: -1198.6155
punishment for action: 2

punishment for action: 3

reward for action: 7

punishment for down
 9685/10000 [============================>.] - ETA: 2s - reward: -1197.7529
reward for right

punishment for action: 9

punishment for action: 6

punishment for action: 


reward for up

reward for action: 6
 9995/10000 [============================>.] - ETA: 0s - reward: -1163.4523
punishment for left

reward for down
10000/10000 [==============================] - 85s 8ms/step - reward: -1162.9418
done, took 423.755 seconds


In [9]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [10]:
import numpy as np
q_table = np.zeros([1000, env.action_space.n])

https://blog.paperspace.com/getting-started-with-openai-gym/

https://github.com/openai/gym/blob/master/gym/spaces/box.py

https://lilianweng.github.io/lil-log/2018/05/05/implementing-deep-reinforcement-learning-models.html